# Lecture 11

In [1]:
import numpy as np
import pandas as pd

# More `EXPLAIN ANALYZE`

In [2]:
%reload_ext sql
%sql postgresql://127.0.0.1:5432/imdb_perf_lecture
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [11]:
%%sql
EXPLAIN ANALYZE
SELECT *
FROM actor, cast_info, movie
WHERE actor.id = cast_info.person_id
  AND movie.id = cast_info.movie_id
LIMIT 10;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

14 rows affected.

QUERY PLAN
Limit (cost=0.86..9.45 rows=10 width=56) (actual time=0.044..0.118 rows=10 loops=1)
-> Nested Loop (cost=0.86..1900819.33 rows=2211936 width=56) (actual time=0.042..0.116 rows=10 loops=1)
-> Nested Loop (cost=0.43..895381.74 rows=2211936 width=26) (actual time=0.032..0.083 rows=10 loops=1)
-> Seq Scan on cast_info (cost=0.00..31907.36 rows=2211936 width=8) (actual time=0.011..0.013 rows=10 loops=1)
-> Memoize (cost=0.43..0.47 rows=1 width=18) (actual time=0.006..0.006 rows=1 loops=10)
Cache Key: cast_info.person_id
Cache Mode: logical
Hits: 2 Misses: 8 Evictions: 0 Overflows: 0 Memory Usage: 1kB
-> Index Scan using actor_pkey on actor (cost=0.42..0.46 rows=1 width=18) (actual time=0.006..0.006 rows=1 loops=8)
Index Cond: (id = cast_info.person_id)


In [12]:
%sql CREATE INDEX actoridindex ON actor(id);

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

++
||
++
++

In [13]:
%%sql
EXPLAIN ANALYZE
SELECT *
FROM actor, cast_info, movie
WHERE actor.id = cast_info.person_id
  AND movie.id = cast_info.movie_id
LIMIT 10;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

14 rows affected.

QUERY PLAN
Limit (cost=0.86..9.45 rows=10 width=56) (actual time=0.087..0.220 rows=10 loops=1)
-> Nested Loop (cost=0.86..1900819.33 rows=2211936 width=56) (actual time=0.086..0.218 rows=10 loops=1)
-> Nested Loop (cost=0.43..895381.74 rows=2211936 width=26) (actual time=0.075..0.189 rows=10 loops=1)
-> Seq Scan on cast_info (cost=0.00..31907.36 rows=2211936 width=8) (actual time=0.025..0.027 rows=10 loops=1)
-> Memoize (cost=0.43..0.47 rows=1 width=18) (actual time=0.015..0.015 rows=1 loops=10)
Cache Key: cast_info.person_id
Cache Mode: logical
Hits: 2 Misses: 8 Evictions: 0 Overflows: 0 Memory Usage: 1kB
-> Index Scan using actoridindex on actor (cost=0.42..0.46 rows=1 width=18) (actual time=0.018..0.018 rows=1 loops=8)
Index Cond: (id = cast_info.person_id)


In [14]:
%sql CREATE INDEX movieid_castinfoindex ON cast_info(movie_id);

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

++
||
++
++

In [15]:
%%sql
EXPLAIN ANALYZE
SELECT *
FROM actor, cast_info, movie
WHERE actor.id = cast_info.person_id
  AND movie.id = cast_info.movie_id
LIMIT 10;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

14 rows affected.

QUERY PLAN
Limit (cost=0.86..6.74 rows=10 width=56) (actual time=0.074..0.340 rows=10 loops=1)
-> Nested Loop (cost=0.86..1299808.98 rows=2211936 width=56) (actual time=0.073..0.337 rows=10 loops=1)
-> Nested Loop (cost=0.43..436334.61 rows=2211936 width=38) (actual time=0.036..0.115 rows=10 loops=1)
-> Seq Scan on movie (cost=0.00..11425.53 rows=656453 width=30) (actual time=0.010..0.011 rows=3 loops=1)
-> Index Scan using movieid_castinfoindex on cast_info (cost=0.43..0.60 rows=5 width=8) (actual time=0.010..0.032 rows=3 loops=3)
Index Cond: (movie_id = movie.id)
-> Memoize (cost=0.43..0.47 rows=1 width=18) (actual time=0.021..0.021 rows=1 loops=10)
Cache Key: cast_info.person_id
Cache Mode: logical
Hits: 0 Misses: 10 Evictions: 0 Overflows: 0 Memory Usage: 2kB


In [16]:
%sql drop index actoridindex;
%sql drop index movieid_castinfoindex;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

++
||
++
++

# Structural Transformation: From Relations to Matrices and Back
- Matrix $\rightarrow$ Relational works.
- Relational $\rightarrow$ Matrix sometimes works!
- But how?

To start, let's take our matrix in `mm.txt`, and load it into Pandas.

In [2]:
import pandas as pd

In [3]:
mm = pd.read_csv('data/mm.txt', header=0)
mm

,Year,OCT,NOV,DEC,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP
0,2002,0.0,6.03,7.18,0.82,2.01,9.96,4.74,0.78,0.13,0.0,0.09,0.0
1,2003,0.0,6.03,7.18,0.82,2.01,9.96,4.74,0.78,0.13,0.0,0.09,0.0
2,2004,0.0,6.03,7.18,0.82,2.01,9.96,4.74,0.78,0.13,0.0,0.09,0.0
3,2005,0.0,6.03,7.18,0.82,2.01,9.96,4.74,0.78,0.13,0.0,0.09,0.0
4,2006,0.0,6.03,7.18,0.82,2.01,9.96,4.74,0.78,0.13,0.0,0.09,0.0
5,2007,0.0,6.03,7.18,0.82,2.01,9.96,4.74,0.78,0.13,0.0,0.09,0.0
6,2008,0.0,6.03,7.18,0.82,2.01,9.96,4.74,0.78,0.13,0.0,0.09,0.0
7,2009,0.0,6.03,7.18,0.82,2.01,9.96,4.74,0.78,0.13,0.0,0.09,0.0
8,2010,0.0,6.03,7.18,0.82,2.01,9.96,4.74,0.78,0.13,0.0,0.09,0.0
9,2011,0.0,6.03,7.18,0.82,2.01,9.96,4.74,0.78,0.13,0.0,0.09,0.0


## What does an unpivot look like (Matrix -> Relational)?

In [7]:
mm_melted = mm.melt(id_vars=['Year'])
mm_melted

,Year,variable,value
0,2002,OCT,0.0
1,2003,OCT,0.0
2,2004,OCT,0.0
3,2005,OCT,0.0
4,2006,OCT,0.0
...,...,...,...
223,2016,SEP,0.0
224,2017,SEP,0.0
225,2018,SEP,0.0
226,2019,SEP,0.0


Thanks to the `id_var` parameter, the `Year` column is named and repeated for all other (variable=column name, value=value) elements in the row.

In [ ]:
mm_melted[mm_melted['Year'] == 2002]

## PIVOT(UNPIVOT) = ??

In [8]:
#mm_melted.pivot(index='variable', columns='Year')
mm_melted.pivot(index='Year', columns='variable')

value                                                               
variable   APR   AUG   DEC   FEB   JAN  JUL   JUN   MAR   MAY   NOV  OCT  SEP
Year                                                                         
2002      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2003      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2004      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2005      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2006      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2007      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2008      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2009      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2010      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2011      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2012      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2013      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2014      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2015      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2016      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2017      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2018      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2019      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0
2020      4.74  0.09  7.18  2.01  0.82  0.0  0.13  9.96  0.78  6.03  0.0  0.0

### Extra Columns
Let's go back to `mmp.txt`. 
- Matrix or relation? 
- Try doing some PIVOT/UNPIVOT work on this.

In [ ]:
mmp = pd.read_csv('data/mmp.txt', header=0)
mmp

In [ ]:
# Unpivot
mmp_melted = mmp.melt(id_vars=['Location', 'Station Name', 'Year'])
mmp_melted

In [ ]:
# Repivot the unpivot

# mmp_melted.pivot(index='Year', columns='variable')
mmp_tt = mmp_melted.pivot(index=['Location', 'Station Name', 'Year'], columns='variable')
mmp_tt

In [ ]:
mmp_tt.reset_index()

# [Extra] Multisets to Sets

### Set up connections and schema

In [ ]:
%reload_ext sql
%sql postgresql://127.0.0.1:5432/postgres
import pandas as pd

In [ ]:
%%sql
drop table if exists blue;
drop table if exists red;
create table blue (last text, first text);
create table red (last text, first text);

insert into blue values ('Wang', 'Daisy');
insert into blue values ('Wang', 'Daisy');
insert into blue values ('Wang', 'Xin');

insert into red values ('Wang', 'Daisy');
insert into red values ('Wang', 'Xin');
insert into red values ('Wang', 'Xin');

select * from blue;

In [ ]:
%sql select * from red;

## Representing multiset relations as counted-set relations

### Use a CTAS statement with group by to convert standard tables to counted-set tables

In [ ]:
%%sql
drop table if exists bluem;
create table bluem as 
select *, count(*) as multiplicity
  from blue
 group by last, first;

select * from bluem;

In [ ]:
%%sql
drop table if exists redm;
create table redm as 
select *, count(*) as multiplicity
  from red
 group by last, first;

select * from redm;

### How do we make selection on counted-set tables work like multisets?

This works exactly the same in both cases. There's nothing special here. Applying `WHERE` filters on a counted-set will always yield a set back, because you're only removing rows from a set. By definition, this cannot create an entity that is not a set. 

In [ ]:
%%sql
-- sigma on multiset
select * from blue 
 where first = 'Daisy';

In [ ]:
%%sql
-- sigma on counted set
select * from bluem where first = 'Daisy';

### What about projection?

We might want to be a bit careful here. See, what defines a set uniquely is its key, and in this case, the key is the combination of `(last, first)`. Simply having `last` or just having `first` is not enough to uniquely identify a row.

In [ ]:
%%sql
-- pi on multiset
select last from blue;

In fact, you can see that if you simply selected `last` from a counted-set, you'd get a multi-set as your output.

In [ ]:
%%sql
select last from bluem;

To convert this to a counted-set again, you need to sum up the multiplicities of the tuples that the `last` field came from.

In [ ]:
%%sql
-- pi on counted set
select last, SUM(multiplicity) from bluem group by last;

### What about cross-product?

In [ ]:
%%sql
-- x on multiset
select * from blue, red;

Next, convert *the output* of a multiset cross-product to a counted set as we did before. This is our desired result:

In [ ]:
%%sql
-- convert multiset x to counted set
  with cte(blast, bfirst, rlast, rfirst)
    as (select * from blue, red)
select *, count(*)
  from cte
 group by blast, bfirst, rlast, rfirst;

Now, what went on in the arithmetic here? We can think this through by pushing the arithmetic into the query!

First, what do you get with naive cross-product of counted sets? You get the names from each table, along with the number of times that each name showed up in its respective table. So, for example, `('Wang', 'Xin')` showed up once in `blue` and twice in `red`.

In [ ]:
%%sql
select * from bluem, redm;

What does each row tell us individually? Each row tells us the number of times that the name from the left must be matched with the name from the right in the original cross product between `blue` and `red`. So if you multiply the multiplicities together, you'll get the number of instances of each ordered pair of names in the final cross product

In [ ]:
%%sql
-- fix multiplicity per row
select b.last, b.first, r.last, r.first, b.multiplicity*r.multiplicity from bluem b, redm r;

If we simply wanted to drop duplicates instead of monitoring how many there were (the point of a counted-set), our life would have been a lot easier...

In [ ]:
%%sql
select distinct b.last, b.first, r.last, r.first
  from blue b, red r;

# [Scratch] Transposing Demo Code

In [ ]:
%reload_ext sql
%sql postgresql://127.0.0.1:5432/postgres
import pandas as pd

In [ ]:
%%sql
drop table if exists example;
create table example(name text, age integer, gpa float);
insert into example values
       ('Patty Perfect', 22, 4.0),
       ('Sameer Soclose', 20, 3.99),
       ('Jacob Excellent', 21, 3.93);

In [ ]:
df = %sql select * from example;
df = df.DataFrame()

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
dft = df.transpose()
dft

In [ ]:
dft.dtypes

In [ ]:
df2 = df.transpose().transpose()
df2

In [ ]:
df2.dtypes

In [ ]:
df2['age'] = df2['age'].astype(int)
df2['gpa'] = df2['gpa'].astype(float)

In [ ]:
df2.dtypes

In [ ]:
mat = np.array([[1, 0, 0, 1, 20000],
          [0, 1, 0, 2, 10011],
          [0, 0, 1, 3, 50000],
          [0, 0, 1, 3, 10000]])
mat

In [ ]:
df = pd.DataFrame({'CompanyName': ['VW', 'Acura', 'Hona', 'Honda'],
              'Categorical Value': [1, 2, 3, 3],
              'Price': [20000, 10011, 50000, 10000]})
df

In [ ]:
df.dtypes

In [ ]:
# %sql --persist df